In [1]:
# MLB spec code for Getting the MLB MVP Props data from SportData as well as the base stats 
# 1. Google Sheets (for Review)
# 2. SQL for Use 

# The Google Sheet for the data review is: 
# MLBGetProps

# https://docs.google.com/spreadsheets/d/1MvVT67sxzg2yBhHNMQXeRMxWQ4D4CnieMI9KfVoya-s/edit

In [2]:
#Packages
#Time
import time
from datetime import datetime, timedelta, timezone, date
from pytz import timezone as tz
#Set Timezone to Eastern time
ny = tz("America/New_York")
fmt = '%B %d, %Y %H:%M:%S'
timestart = datetime.now(ny)

import sys, os

#UrlLib
import urllib.request
from urllib.request import Request, urlopen
import json 

#Gspread
from gspread import authorize
from gspread_dataframe import get_as_dataframe, set_with_dataframe
from oauth2client.service_account import ServiceAccountCredentials

#SQL
from os import environ
from sqlalchemy import create_engine

#Beatiful Soup
from bs4 import BeautifulSoup


#Pandas
from pandas import read_sql,DataFrame, read_html, read_json , merge, concat,to_numeric, set_option, json_normalize


#SQL
from os import environ
import sqlalchemy
from sqlalchemy import create_engine


import re
import requests

#For Sims:

from df2gspread import df2gspread as d2g
import warnings
import random
import re


In [3]:
#defining header to use on URL requests
header= {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) ' 
      'AppleWebKit/537.11 (KHTML, like Gecko) '
      'Chrome/23.0.1271.64 Safari/537.11',
      'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
      'Accept-Charset': 'ISO-8859-1,utf-8;q=0.7,*;q=0.3',
      'Accept-Encoding': 'none',
      'Accept-Language': 'en-US,en;q=0.8',
      'Connection': 'keep-alive'}

In [4]:
#Connect to Debug GSheet
try:
    link='https://api.sportradar.us/oddscomparison-player-props/production/v2/en/players/mappings.json?api_key=gkppjuantd4fnh54b4mnhzbf&start='
    scope = ['https://spreadsheets.google.com/feeds', 'https://www.googleapis.com/auth/drive']
    credentials = ServiceAccountCredentials.from_json_keyfile_name('creds.json', scope) 
    gc = authorize(credentials)
    timenow = datetime.now(ny)
    sheet = gc.open('MLBGetProps')
    Goal = sheet.worksheet("Scrape")
    Goal.update('b8', timestart.strftime("%B %d, %Y %H:%M:%S"))
except:
    timenow = datetime.now(ny)


In [5]:
#Get Relevant Season Stats
#Get MVP odds from SportData

print('Get Season Stats from SportData')
dataStats = []

#Get Rosters FantasyData
with urlopen("https://api.sportsdata.io/v3/mlb/stats/json/PlayerSeasonStats/2023?key=9080515fbdff4aa095490cadda5ecd30") as url:
    data = json.loads(url.read().decode())
for players in data:
    row=[ players['PlayerID'],players['HomeRuns'],players['OnBasePercentage'],players['SluggingPercentage'],players['OnBasePlusSlugging'] ]
    dataStats.append(row)
    

dfStats=DataFrame(dataStats)
dfStats.columns = ["SD_ID", "HR", "OBP", "SLG", "OPS"]

#Let's authorize us to work on that file
timestart =  datetime.now()
sheet = gc.open('MLBGetProps')

#Let's push the map
Goal = sheet.worksheet("Stats")
sheet.values_clear("Stats!a1:e2000")

set_with_dataframe(Goal, dfStats, row=1,col=1)
timenow = datetime.now(ny)
Goal.update('f2', timenow.strftime("%B %d, %Y %H:%M:%S"))


Get Season Stats from SportData


{'spreadsheetId': '1MvVT67sxzg2yBhHNMQXeRMxWQ4D4CnieMI9KfVoya-s',
 'updatedRange': 'Stats!F2',
 'updatedRows': 1,
 'updatedColumns': 1,
 'updatedCells': 1}

In [7]:
#Get MVP odds from SportData

print('Get MVP odds from SportData')
dataNL = []
dataAL = []

#Get Rosters FantasyData
with urlopen("https://api.sportsdata.io/v3/mlb/odds/json/BettingFuturesBySeason/2023?key=dc0a09ab9b2f412492a939da443252e5") as url:
    data = json.loads(url.read().decode())
    bets= data[0]['BettingMarkets']
for bet in bets:
    bettingoutcomes= bet['BettingOutcomes']
    bettype=bet['BettingBetType']
    if bettype=='NL MVP':
        for betoutcome in bettingoutcomes:
            if betoutcome['SportsBook']['Name'] == 'FanDuel':
                row=[ bettype, betoutcome['SportsBook']['Name'], betoutcome['PlayerID'], betoutcome['Participant'],betoutcome['PayoutAmerican']]
                dataNL.append(row)
    if bettype=='AL MVP':
        for betoutcome in bettingoutcomes:
            if betoutcome['SportsBook']['Name'] == 'FanDuel':
                row=[ bettype, betoutcome['SportsBook']['Name'], betoutcome['PlayerID'], betoutcome['Participant'],betoutcome['PayoutAmerican']]
                dataAL.append(row)

dfNL=DataFrame(dataNL)
dfNL.columns = ["Bet", "Book", "SD_ID", "Player", "MoneyLine"]
dfAL=DataFrame(dataAL)
dfAL.columns = ["Bet", "Book", "SD_ID", "Player", "MoneyLine"]

dfNL=dfNL.sort_values(by=['MoneyLine'])
dfAL=dfAL.sort_values(by=['MoneyLine'])

combinedNL= dfNL.merge(dfStats, on='SD_ID')
combinedAL= dfAL.merge(dfStats, on='SD_ID')

#Let's authorize us to work on that file
timestart =  datetime.now()
sheet = gc.open('MLBGetProps')

#Let's push the map
Goal = sheet.worksheet("MVPNL")
sheet.values_clear("MVPNL!a1:i2000")

set_with_dataframe(Goal, combinedNL, row=1,col=1)
timenow = datetime.now(ny)
Goal.update('j2', timenow.strftime("%B %d, %Y %H:%M:%S"))

Goal = sheet.worksheet("MVPAL")
sheet.values_clear("MVPAL!a1:i2000")

set_with_dataframe(Goal, combinedAL, row=1,col=1)
timenow = datetime.now(ny)
Goal.update('j2', timenow.strftime("%B %d, %Y %H:%M:%S"))


Get MVP odds from SportData


{'spreadsheetId': '1MvVT67sxzg2yBhHNMQXeRMxWQ4D4CnieMI9KfVoya-s',
 'updatedRange': 'MVPAL!J2',
 'updatedRows': 1,
 'updatedColumns': 1,
 'updatedCells': 1}